<a href="https://colab.research.google.com/github/Agniswar123/Simplified-DES/blob/main/Simple%20DES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
#S-DES Encryption
#Ref: http://mercury.webster.edu/aleshunas/COSC%205130/G-SDES.pdf
#     https://www.youtube.com/watch?v=ewHnHWCsFNM
#Example:

#PT:[1,0,0,1,0,1,1,1] 
#CT:[0, 0, 1, 1, 1, 0, 0, 0]

#For encryption or Decryption, code upro Key generated is required

In [156]:
plain_text=[0,1,1,1,0,0,1,0]    
key=[1,0,1,0,0,0,0,0,1,0]  
cipher_text=[0,1,1,1,0,1,1,1]      

In [134]:
#---------------------------------------------Key Generation----------------------------------#

In [135]:
#length check
if len(key) > 10 or len(plain_text) > 8:
  exit(0)

In [136]:
p10_table=[3,5,2,7,4,10,1,9,8,6]
kout=[]
for i in range(len(p10_table)):
  kout.append(int(key[p10_table[i]-1]))
print(kout)

[1, 0, 0, 0, 0, 0, 1, 1, 0, 0]


In [137]:
k1left=kout[:5]
k1right=kout[5:]
k2left=kout[:5]
k2right=kout[5:]

In [138]:
print(k1left,k1right,k2left,k2right)

[1, 0, 0, 0, 0] [0, 1, 1, 0, 0] [1, 0, 0, 0, 0] [0, 1, 1, 0, 0]


In [139]:
#one left shift to get k1
k1left.append(k1left.pop(0))
k1right.append(k1right.pop(0))

print(k1left,k1right)

#three total left shift to get k2
for i in range(3):
  k2left.append(k2left.pop(0))
  k2right.append(k2right.pop(0))
  print(k2left, k2right)

[0, 0, 0, 0, 1] [1, 1, 0, 0, 0]
[0, 0, 0, 0, 1] [1, 1, 0, 0, 0]
[0, 0, 0, 1, 0] [1, 0, 0, 0, 1]
[0, 0, 1, 0, 0] [0, 0, 0, 1, 1]


In [140]:
k5=k1left+k1right
k6=k2left+k2right

In [141]:
p8_table=[6,3,7,4,8,5,10,9]
K1=[]
K2=[]
for i in range(len(p8_table)):
  K1.append(k5[p8_table[i]-1])
  K2.append(k6[p8_table[i]-1])

In [142]:
print(K1,K2)

[1, 0, 1, 0, 0, 1, 0, 0] [0, 1, 0, 0, 0, 0, 1, 1]


In [143]:
#--------------------------------------Key Generated------------------------------------------#

In [144]:
#---------------------------------------Encryption--------------------------------------------#

In [145]:
def Position(a=0, b=0):
  return a*2+b

def Binary(a=0):
  if a==0:
    return '0 0 '
  elif a==1:
    return '0 1 '
  elif a==2:
    return '1 0 '
  elif a==3:
    return '1 1 '

In [146]:
def InitialPermutation(plain_text):       
  ip8_table=[2,6,3,1,4,8,5,7]
  plain_out1=[]
  for i in range(len(ip8_table)):
    plain_out1.append(int(plain_text[ip8_table[i]-1]))
  
  return plain_out1

In [147]:
def ExpandedPermutation(text):      
  ep_table=[4,1,2,3,2,3,4,1]
  expanded=[]
  for i in range(len(ep_table)):
    expanded.append(text[ep_table[i]-1])
  return expanded

In [148]:
def Permutation4(text):
  p4_table=[2,4,3,1]
  perm_out=[]
  for i in range(4):
    perm_out.append(text[p4_table[i]-1])
  return perm_out

In [149]:
def IPInverse(text):
  ipinv=[4,1,3,5,7,2,8,6]
  ip=[]
  for i in range(8):
    ip.append(text[ipinv[i]-1])
  return ip

In [150]:
#XOR of two arrays
def Xor(ar1,ar2):          
  xor_out=[]
  for i in range(len(ar1)):
    xor_out.append(ar1[i] ^ ar2[i])
  return xor_out

In [151]:
#xor_left for S0 and xor_right for S1

def SBox(S0,S1,xor_left,xor_right):
  s0_row = Position(xor_left[0], xor_left[-1])
  s0_col = Position(xor_left[1], xor_left[-2])

  s1_row = Position(xor_right[0], xor_right[-1])
  s1_col = Position(xor_right[1], xor_right[-2])

  out=str(Binary(S0[s0_row][s0_col])+Binary(S1[s1_row][s1_col])).split(" ")
  temp=out.pop()
  out=[int(x) for x in out]
  return out

In [152]:
S0 = [
      [1,0,3,2],
      [3,2,1,0],
      [0,2,1,3],
      [3,1,3,2]
]

S1 = [
      [0,1,2,3],
      [2,0,1,3],
      [3,0,1,0],
      [2,1,0,3]
]

In [153]:
def FunctionK(text,key):
  #splitting in two halves
  plain_out1_left = text[:4]
  plain_out1_right = text[4:]

  #Expansion
  right_expanded=ExpandedPermutation(plain_out1_right)
  #print(right_expanded)

  #Xor with Key
  xor_K1 = Xor(right_expanded,key)
  #print(xor_K1)

  #Splitting
  xor_K1_left=xor_K1[:4]
  xor_K1_right=xor_K1[4:]

  #SBoxing
  perm = SBox(S0, S1, xor_K1_left, xor_K1_right)

  #Permutation
  perm_out = Permutation4(perm)
  perm_xor=Xor(perm_out, plain_out1_left)
  #print(perm_out,perm_xor)
  temp = perm_xor + plain_out1_right
  return temp

In [154]:
#Initial Permutation
plain_out1=InitialPermutation(plain_text)

#Round 1 of fk
round1= FunctionK(plain_out1, K1)

#Switching
round2_input=round1[4:]+round1[:4]

#Round 2 of fk
round2= FunctionK(round2_input, K2)

#IPInverse
cipher_text= IPInverse(round2)

print(cipher_text)

[0, 1, 1, 1, 0, 1, 1, 1]


In [155]:
#---------------------------------Decryption--------------------------

In [157]:
#Initial Permutation
decrypt=InitialPermutation(cipher_text)

#Round 1 of fk
reverse1= FunctionK(decrypt, K2)

#Switching
reverse2_input= reverse1[4:]+reverse1[:4]

#Round 2 of fk
reverse2= FunctionK(reverse2_input, K1)

#IPInverse
decrypted_msg= IPInverse(reverse2)

print(decrypted_msg)

[0, 1, 1, 1, 0, 0, 1, 0]
